Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model then estimates the parameters using simulated annealing

In [1]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters


using DataFrames
using BenchmarkTools



In [2]:
using NBInclude
@nbinclude("solve_model.ipynb")

#importou a função solve_tree

solve_tree (generic function with 5 methods)

Começando o simulated annealing

In [3]:
function error_vector(data_moments, simulated_moments)
    #two arrays with the data moments and the simulated moments
    
    return (simulated_moments .- data_moments) ./ data_moments
    
end



    

error_vector (generic function with 1 method)

In [4]:
function criterion(S, N, data, W_hat, game_parameters, simulation_parameters, s_W, j_W, Pst_array, Pjt_array)
    
    θs0, θj0, μ = simulation_parameters
    θs0 = Int64(θs0)
    θj0 = Int64(θj0)
      
    ρ = game_parameters[1]
    c0 = game_parameters[3]
    λj = game_parameters[4]
    c1 = Float64(0.015) #calibrated


    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = (1.0 - Dj)
    
    D = Dj + Ds
    
    #return error message if the data is not escaled
    #retorna mensagem de erro se os dados não estiverem em escala
    @assert(Ds > 0 && D == 1.0)
    
    
    function simulate_moments(S, N, θs0, θj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array)
    
    
        Results = loop_simulations(S, N, θs0, θj0, μ, λj)

        #S simulations, 8 moments
        Moments = zeros(S, 8)


        for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.Dj = Results[s,:,6];
            DF.Ds = Results[s,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = INCOURT
            mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT
            mm5.payoff_s = mm5.payoff_s ./ PRECOURT.Ds
            mm5 = mean(mm5.payoff_s)


            #6. avg recovery rate for junior given precourt

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT
            mm6.payoff_j = mm6.payoff_j ./ PRECOURT.Dj
            mm6 = mean(mm6.payoff_j)


            #7. junior avg fraction gain given incourt

            mm7 = INCOURT
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt

            mm8 = INCOURT
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)



            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]

        end

        return vec(mean(Moments, dims=1))
    end

    
    
    function loop_simulations(S, N, θs0, θj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array)
    
        #S is the number of simulations
        #N is the number of observations

        #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster

        Results = zeros(S, N, 7)

        #últimas entradas de Results são os valores das dívidas
        Results[:,:, end-1] .= Dj
        Results[:,:, end] .= Ds


        for s in 1:S
            for n in 1:N
                Results[s, n, 1:5] .= simulate_game(θs0, θj0, μ, λj)
            end
        end

        return Results

    end
    
    
    
    function simulate_game(Hs0, Hj0, μ, λj, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
        hst = Hs0
        hjt = Hj0

        #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
        lst = hst
        ljt = hjt

        result = zeros(5);

        #number of observed proposals
        observed_proposals = 0.0


        #recovering "T+1" from s_W 
        T = size(s_W,1)


        while(result[1]==0.0 && t < T)


            u = rand()

            if(u < λj)
                propositor = "j"

            else
                propositor = "s"

            end


            #setting the default variables according to the propositor
            Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt)


            #proposal ####
            policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

            #lm_next is the update of the adversary's lower bound
            
            #lm_next==grid+1 means the payment offer is greater than the adversary continuation value. 
            #but in answer_reorg, the adversary searches his continuation value using lm_next. So it cannot be grid+1
            #lm_next==grid+2 means a waiting offer, thus we do not update the lower bound
            #policy!= 3.0 means either a waiting offer or a liquidation offer. In both cases we do not update lmt
            if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
                lm_next = lmt
            else
                lm_next = Int64(lm_next)
            end



            if(policy==3.0)

                observed_proposals += 1.0

                payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


                if(answer==1.0)

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals

                else

                    t+=1

                    hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
                end

            elseif(policy==2.0)

                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)



            else      
                #(policy==1.0)

                observed_proposals += 1.0

                payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop)

                if(answer==1.0)

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 1.0
                    result[4] = t
                    result[5] = observed_proposals

                else

                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals


                end
            end


            if(t==T)

                result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
                result[2] = j_W[T, 1, 1, 1]
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            end





        end


        return result
    end
    
    function choose_parameters(propositor, hst, lst, hjt, ljt)
    
        if(propositor=="s")

            Pkt_array = Pst_array
            Cont_val = j_W
            prop_index = 1
            respondent_index = 2 #índice de j, para organizar o payoff
            m_L = j_L

            hkt = hst
            lkt = lst

            hmt = hjt
            lmt = ljt


            lk_next = hkt

            hk_next = draw_beta(hkt)
            hm_next = draw_beta(hmt)

        elseif(propositor=="j")
            Pkt_array = Pjt_array
            Cont_val = s_W
            prop_index = 2
            respondent_index = 1
            m_L = s_L

            hkt = hjt
            lkt = ljt

            hmt = hst
            lmt = lst


            lk_next = hkt


            hk_next = draw_beta(hkt)
            hm_next = draw_beta(hmt)

        else

            println("error: propositor not valid")

        end

        return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
    end


    function draw_beta(hkt, β=game_parameters[2], grid=100)

        u = rand()

        if(hkt == grid)

            return grid

        else

            x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
            x = round(x * grid, digits=0)
    #         return Int64(x * 100) #to convert in an integer
            return Int64(x)

        end
    end
    
    
    #cost function
    function Ct(t)
        #cost at period t=0(index1) is 0
        if(t <= 1)
            return 0
        else
            return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
        end
    end



    #liquidation payoffs
    function s_L(t)
        return min(L - Ct(t), Ds)
    end


    function j_L(t)
        return min(L - Ct(t) - s_L(t), Dj)
    end


    function proposal(Pkt_array, t, hkt, lmt)

        #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação

        return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]

    end


    #maximum value of reorganization each period
    function Vt(Vmax, ρ, t)

        if(t <=1)
            return Vmax
        else
            #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
            return ρ^(t-2) * Vmax
        end


    end

    # answer_liq
    function answer_liq(m_L, t, hm_next, payoff_prop)

        liq = (m_L(t), Vt(Vmax, ρ, t) * hm_next/100 - payoff_prop)

        payoff_liq, answer = findmax(liq)

        return payoff_liq, answer
    end

    function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)

        reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

        payoff_reorg, answer = findmax(reorg_value)

        return payoff_reorg, answer
    end

    function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)

        if(propositor=="s")
            hst = hk_next
            lst = lk_next

            hjt = hm_next
            ljt = lm_next

        else
            hst = hm_next
            lst = lm_next

            hjt = hk_next
            ljt = lk_next
        end


        return hst, lst, hjt, ljt
    end

    
    #simulation####
    simulated_moments = simulate_moments(S, N, θs0, θj0, μ, λj, s_W, j_W, Pst_array, Pjt_array)
    
    #inputed here manually
    data_moments = [0.85, 0.24, 2.78, 0.95, 1.00, 1.00, 0.75, 0.65]
    
    err = error_vector(data_moments, simulated_moments)
    
    #* in this case represents matrix multiplication
    criterion_val = transpose(err) * W_hat
    criterion_val = criterion_val * err
    
#     return (transpose(err) * W_hat) * err
    return criterion_val
    
end



    
    
    

criterion (generic function with 1 method)

Simulated Annealing

In [5]:
function mu_inv(y, mu)
    
    
    #entender porque tem 1 somando em mu
    #entender porque tem 1 subtraindo de abs(y)
    x = (((1 .+ mu) .^ abs.(y) .- 1)/mu) .* sign.(y)
    
    return x
    
end 

mu_inv (generic function with 1 method)

In [6]:
function compare_solutions(f0, g0, s0, fc, g, s, ftest, gtest, stest, Temp=Temp, TolFun=TolFun)
    
    
    #f0: function before the current loop
    #fc: function evaluated at the best current solution inside the loop
    #ftest: function evaluated at the test point inside the loop
    #gtest: game_parameters, test
    #stest: simulation_parameters, test
    
    #we evaluate the difference between the test point (ft) and the current point inside the loop(fc)
    #at the end, we compare the best between ftest and fc to the function before the loop (f0)
    
    df=ftest-fc

    #If the function variation,df, is <0 we take test point as current
    #point. And if df>0 we use Metropolis [5] condition to accept or
    #reject the test point as current point.
    #We use eps and TolFun to adjust temperature [4].   

    #retirei eps porque não estava definido antes. Cadê o cabo azul?
    if ((df < 0 || rand() < exp(-Temp*df/(abs(fc))/TolFun))==true)
        g = gtest
        s = stest
        fc = ftest 
    end

    #If the test point's solution (fx1) is better than current solution (fx), we take
    #current point as cuyrrent solution.       
    if ((fc < f0) ==true)
        g0 = gtest
        s0 = stest
        f0 = ftest
    end
    
    
    return g, s, fc, g0, s0, f0
    
end
    

compare_solutions (generic function with 3 methods)

In [7]:
function sim_anl(f, S, N, data, W_hat, g0, s0, lg, ug, ls, us, Mmax, ntests, TolFun)
    
    #algorithm adapted from 
    #https://www.mathworks.com/matlabcentral/fileexchange/33109-simulated-annealing-optimization
    
    #inputs####
    #f : function to evaluate
    #S: number of simulations per observation
    #N: number of observations in the data set
    #data = [Vh/D, L/D, Dj/D]
    #W_hat: weighting matrix for criterion function
    #g0: initial game parameters
    #s0: initial simulation parameters
    #lg and ug: lower and upper bounds for game parameters
    #ls and us: lower and upper bounds for simulation parameters
    #Mmax: maximum number of temperatures
    #ntest: number of test points simulated for each temperature
    #TolFun: parameter of tolerance to help choose the appropriate solutions
    
    
    #outputs####
    #the optimal values of the function and its parameters: f0, g0, s0
    #the best value of the function at each temperature and its parameters: fvals, gvals, svals
    
    
    
    
    if (size(g0,1)+size(s0,1))<6
        TolFun=1e-4;
        
    elseif (size(g0,1)+size(s0,1))<5
        Mmax=100;
    end

    
    
    
    #inicializando o algoritmo
    
    g = g0
    s = s0
    
    s_W, j_W, Pst_array, Pjt_array = solve_tree(data, g)
    
    fc = f(S, N, data, W_hat, g, s, s_W, j_W, Pst_array, Pjt_array)
    f0 = fc
    
    #para guardar os valores
    fvals = zeros(Mmax)
    gvals = zeros(Mmax, size(g, 1))
    svals = zeros(Mmax, size(s, 1))


    #Main loop simulates de annealing from a high temperature to zero in Mmax iterations.
    for m in 1:Mmax
        #We calculate T as the inverse of temperature.
        #Boltzman constant = 1
        Temp = m/Mmax
        mu=10^(Temp*100)
        
        
        #message so that we can see the upgrade
        print("cycle = $m", "g = $g", "s = $s")


        #For each temperature we take 500 test points to simulate reach termal
        #equilibrium.
        for t1 in 1:ntests
            
            
            #we generate new gtest points using mu_inv function
            dg = mu_inv(2 .* rand(size(g,1)) .- 1, mu) .* (ug .- lg)
            
            dg = round.(dg, digits=3)
            
            gtest = g .+ dg
            
            #next step is to keep solution within bounds
            gtest = (gtest .< lg) .* lg .+ (lg .<= gtest) .* (gtest .<= ug) .* gtest .+ (ug .< gtest) .* ug
            
            #we evaluate the function and the change between the test point and the current point
            #for this we will have to solve the game, since we changed the game_parameters
            s_W, j_W, Pst_array, Pjt_array = solve_tree(data, gtest)
            
            #now we start another loop to make changes in s, the simulation parameters
            for t2 in 1:(ntests*100) #the cost to test simulaton parameters is lower
                
                
                #we generate new stest points using mu_inv function
                ds = mu_inv(2 .* rand(size(s,1)) .- 1, mu) .* (us .- ls)
                
                #we need to round because these are changes in θs0, θj0, which will be used as integers
                ds[1:2] .= round.(ds[1:2], digits=0)
                
                #ds[3] is μ, which can have 3 digits
                ds[3] = round(ds[3], digits=3)

                stest = s .+ ds

                #next step is to keep solution within bounds
                #this is just a multiplication using indicator functions
                stest = (stest .< ls) .* ls .+ (ls .<= stest) .* (stest .<= us) .* stest .+ (us .< stest) .* us
                

                
                #now we evaluate the criterion function directly
                #we don't need to solve the model again because we didn't change the game parameters
                
                #evaluation the criterion function
                #gtest = game_parameters in this test
                #stest = simulation parameters in this test

                ftest = f(S, N, data, W_hat, gtest, stest, s_W, j_W, Pst_array, Pjt_array)


                #comparing ftest with the current solution fc and the previous solution f0
                g, s, fc, g0, s0, f0 = compare_solutions(f0, g0, s0, fc, g, s, ftest, gtest, stest, Temp, TolFun)
                
            end
        end
        
        
        #record the value of the best solution of each cycle
        fvals[m] = f0
        gvals[m, :] .= g0
        svals[m, :] .= s0
            
        end
    
    return f0, g0, s0, fvals, gvals, svals
    
end





sim_anl (generic function with 1 method)

### 1,2,3, testando

Teste com Mmax = 10, ntests = 5 requer 1 + Mmax * ntests iterações = 51

Se cada iteração demorar 5 min, são 255min, ou 4h15min.

Bora

In [8]:
#testing

f = criterion
S = 40
N = 75

#Vh/D, L/D, Dj/D
data = [1.0, 0.25, 0.68]; 

W_hat = Matrix{Int}(I, 8, 8);

#ρ, β, c0, λj
g0 = [0.884, 9.84, 0.044, 0.346];

#bounds for g
lg = [0.5, 1.0, 0.001, 0.1]
ug = [0.884, 12.0, 0.09, 0.9]


#θs0, θj0, μ
s0 = [28, 36, 4.566]

#bounds for s
#μ can be 0.5 month or 12 months
ls = [1, 1, 0.5]
us = [100, 100, 12]

Mmax = 10
ntests = 5
TolFun = 1e-4

fbest, gbest, sbest, fvals, gvals, svals = sim_anl(f, S, N, data, W_hat, g0, s0, lg, ug, ls, us, Mmax, ntests, TolFun)

328.667608 seconds (635.83 M allocations: 210.125 GiB, 8.89% gc time)
cycle = 1g = [0.884, 9.84, 0.044, 0.346]s = [28.0, 36.0, 4.566]333.992146 seconds (633.50 M allocations: 210.055 GiB, 10.06% gc time)
381.843828 seconds (633.51 M allocations: 210.056 GiB, 10.26% gc time)
355.340801 seconds (633.51 M allocations: 210.056 GiB, 10.53% gc time)
145.152528 seconds (272.98 M allocations: 102.080 GiB, 12.30% gc time)
154.601818 seconds (273.26 M allocations: 102.296 GiB, 12.01% gc time)
cycle = 2g = [0.681, 9.847000000000001, 0.044, 0.597]s = [1.0, 1.0, 3.987999999999999]147.127398 seconds (273.26 M allocations: 102.296 GiB, 12.37% gc time)
145.063138 seconds (273.23 M allocations: 102.314 GiB, 12.47% gc time)
135.917491 seconds (273.26 M allocations: 102.296 GiB, 12.32% gc time)
141.898201 seconds (273.26 M allocations: 102.296 GiB, 12.07% gc time)
135.775050 seconds (273.10 M allocations: 102.222 GiB, 12.00% gc time)
cycle = 3g = [0.681, 9.847000000000001, 0.044, 0.597]s = [1.0, 1.0, 3.9

(2.8323519367753898, [0.652, 9.925, 0.041999999999999996, 0.597], [1.0, 1.0, 3.987999999999999], [3.1292281244681472, 3.126249029026509, 3.111019164980945, 3.111019164980945, 3.111019164980945, 3.1105785266587818, 2.9284661176889135, 2.9126166535515567, 2.85079248086068, 2.8323519367753898], [0.681 9.847000000000001 0.044 0.597; 0.681 9.847000000000001 0.044 0.597; … ; 0.652 9.925 0.041999999999999996 0.597; 0.652 9.925 0.041999999999999996 0.597], [1.0 1.0 3.987999999999999; 1.0 1.0 3.987999999999999; … ; 1.0 1.0 3.987999999999999; 1.0 1.0 3.987999999999999])

In [9]:
@show fbest
@show gbest
@show sbest

fbest = 2.8323519367753898
gbest = [0.652, 9.925, 0.041999999999999996, 0.597]
sbest = [1.0, 1.0, 3.987999999999999]


3-element Array{Float64,1}:
 1.0
 1.0
 3.987999999999999